In [ ]:
from ipywidgets import interact
import tensorflow as tf
session = tf.InteractiveSession()

In [ ]:
from data import decode_output_sequences
from model import Seq2SeqModel

# Date normalization model

In [ ]:
from date_generator import generate_data, SYMBOLS, SYMBOL_TO_IDX, INPUT_SEQ_LEN, OUTPUT_SEQ_LEN

session.close()
tf.reset_default_graph()
session = tf.InteractiveSession()

hidden_units = 128
batch_size = 32
num_symbols = len(SYMBOL_TO_IDX)

with tf.variable_scope('model', reuse=None):
    training_model = Seq2SeqModel(session=session,
                                    hidden_units = hidden_units, 
                                    input_sequence_len = INPUT_SEQ_LEN,
                                    output_sequence_len = OUTPUT_SEQ_LEN,
                                    num_input_symbols = num_symbols,
                                    num_output_symbols = num_symbols,
                                    batch_size = batch_size,
                                    is_training=True)

training_model.init_variables()

with tf.variable_scope('model', reuse=True):
    testing_model = Seq2SeqModel(session=session,
                                    hidden_units = hidden_units, 
                                    input_sequence_len = INPUT_SEQ_LEN,
                                    output_sequence_len = OUTPUT_SEQ_LEN,
                                    num_input_symbols = num_symbols,
                                    num_output_symbols = num_symbols,
                                    batch_size = batch_size,
                                    is_training=False)

print("Finished building model")

Generate some data to look at.

In [ ]:
from date_generator import generate_data, SYMBOLS, SYMBOL_TO_IDX, INPUT_SEQ_LEN, OUTPUT_SEQ_LEN 

date_generator = generate_data(batch_size=10, random_format=False)
x, y = next(date_generator)

input_strings = decode_output_sequences(x, symbols=SYMBOLS)
target_strings = decode_output_sequences(y, symbols=SYMBOLS)

pprint([("Input", "Target")] + 
       list(zip(input_strings, target_strings)))

In [ ]:
date_generator = generate_data(batch_size=32)
training_model.fit(date_generator, num_epochs=20, batches_per_epoch=64)
print("Finished training")

In [ ]:
from pprint import pprint

batch_size = 20
date_generator = generate_data(batch_size=batch_size, random_format=False)

x, y = next(date_generator)
input_strings = decode_output_sequences(x, symbols=SYMBOLS)
target_strings = decode_output_sequences(y, symbols=SYMBOLS)

model_output = testing_model.predict(x)
pred_strings = decode_output_sequences(model_output, symbols=SYMBOLS)

pprint([("Input", "Target", "Output")] + 
       list(zip(input_strings, target_strings, pred_strings)))

# Addition model

Train the model to add two numbers, x+y, and output the sum.

Generate some example data:

In [ ]:
from addition_generator import AdditionGenerator, SYMBOLS, SYMBOL_TO_IDX, INPUT_SEQ_LEN, OUTPUT_SEQ_LEN 

addition_generator = AdditionGenerator(batch_size=3)
x, y = addition_generator.next_batch()

input_strings = decode_output_sequences(x, symbols=SYMBOLS)
target_strings = decode_output_sequences(y, symbols=SYMBOLS)

print(" Inputs:", input_strings)
print("Targets:", target_strings)

In [ ]:
session.close()
tf.reset_default_graph()
session = tf.InteractiveSession()

hidden_units = 128
num_layers = 2
training_batch_size = 32
num_symbols = len(SYMBOL_TO_IDX)

with tf.variable_scope('model', reuse=None):
    training_model = Seq2SeqModel(session=session,
                                    hidden_units=hidden_units, 
                                    num_layers=num_layers,
                                    input_sequence_len = INPUT_SEQ_LEN,
                                    output_sequence_len = OUTPUT_SEQ_LEN,
                                    num_input_symbols = num_symbols,
                                    num_output_symbols = num_symbols,
                                    batch_size=training_batch_size,
                                    is_training=True)

training_model.init_variables()

with tf.variable_scope('model', reuse=True):
    testing_model = Seq2SeqModel(session=session,
                                    hidden_units = hidden_units, 
                                    num_layers=num_layers,
                                    input_sequence_len = INPUT_SEQ_LEN,
                                    output_sequence_len = OUTPUT_SEQ_LEN,
                                    num_input_symbols = num_symbols,
                                    num_output_symbols = num_symbols,
                                    batch_size=training_batch_size,
                                    is_training=False)

addition_generator = AdditionGenerator(batch_size=training_batch_size)

print("Finished building model")

In [ ]:
training_model.fit_curr(addition_generator, testing_model=testing_model, num_epochs=40, batches_per_epoch=20)
print("Finished training")

In [ ]:
from pprint import pprint

batch_size = 10
test_generator = AdditionGenerator(batch_size=batch_size, number_len=3)

x, y = test_generator.next_batch()
input_strings = decode_output_sequences(x, symbols=SYMBOLS)
target_strings = decode_output_sequences(y, symbols=SYMBOLS)

model_output = testing_model.predict(x)
pred_strings = decode_output_sequences(model_output, symbols=SYMBOLS)

print("Error rate:", testing_model.validate([(x, y)]))

pprint([("Input", "Target", "Output")] + 
       list(zip(input_strings, target_strings, pred_strings)))


# Program execution model

In [13]:
from program_generator import ProgramGenerator, SYMBOLS, SYMBOL_TO_IDX, INPUT_SEQ_LEN, OUTPUT_SEQ_LEN 

program_generator = ProgramGenerator(batch_size=10, program_length=1, num_len=2)
x, y = program_generator.next_batch()

input_strings = decode_output_sequences(x, symbols=SYMBOLS)
target_strings = decode_output_sequences(y, symbols=SYMBOLS)

print(" Inputs:", input_strings)
print("Targets:", target_strings)

 Inputs: ['__________________________________________________)a(tnirp\n85-=-a\n23=a', '___________________________________________________)d(tnirp\n5=-d\n78-=d', '___________________________________________________)a(tnirp\n8+a=a\n5-=a', '____________________________________________________)g(tnirp\n0=-g\n1-=g', '___________________________________________________)b(tnirp\n45-b=b\n8=b', '___________________________________________________)c(tnirp\n27-c=c\n4=c', '__________________________________________________)b(tnirp\n93-=+b\n1-=b', '___________________________________________________)f(tnirp\n27=+f\n97=f', '__________________________________________________)a(tnirp\n37+a=a\n2-=a', '___________________________________________________)b(tnirp\n7+b=b\n43=b']
Targets: ['90____', '-92___', '3_____', '-1____', '-46___', '-68___', '-40___', '151___', '71____', '41____']


In [14]:
session.close()
tf.reset_default_graph()
session = tf.InteractiveSession()

hidden_units = 320
num_layers = 2
training_batch_size = 128
num_symbols = len(SYMBOL_TO_IDX)

with tf.variable_scope('model', reuse=None):
    training_model = Seq2SeqModel(session=session,
                                    hidden_units=hidden_units, 
                                    num_layers=num_layers,
                                    input_sequence_len = INPUT_SEQ_LEN,
                                    output_sequence_len = OUTPUT_SEQ_LEN,
                                    num_input_symbols = num_symbols,
                                    num_output_symbols = num_symbols,
                                    batch_size=training_batch_size,
                                    is_training=True)

training_model.init_variables()

with tf.variable_scope('model', reuse=True):
    testing_model = Seq2SeqModel(session=session,
                                    hidden_units = hidden_units, 
                                    num_layers=num_layers,
                                    input_sequence_len = INPUT_SEQ_LEN,
                                    output_sequence_len = OUTPUT_SEQ_LEN,
                                    num_input_symbols = num_symbols,
                                    num_output_symbols = num_symbols,
                                    batch_size=training_batch_size,
                                    is_training=False)

program_generator = ProgramGenerator(batch_size=training_batch_size, program_length=1, num_len=2)

print("Finished building model")

Finished building model


In [ ]:
training_model.fit(program_generator, 
                testing_model=testing_model, 
                num_epochs=20000, 
                batches_per_epoch=128)
print("Finished training")

In [23]:
saver = tf.train.Saver()
saver.restore(session, "trained_model/model_(2, 3).ckpt")

In [28]:
from random import seed

seed(101)
test_generator = ProgramGenerator(batch_size=training_batch_size, num_len=2, program_length=3)

x, y = test_generator.next_batch(validation=True)

input_strings = decode_output_sequences(x, symbols=SYMBOLS)
target_strings = decode_output_sequences(y, symbols=SYMBOLS)

model_output = testing_model.predict(x)

pred_strings = decode_output_sequences(model_output, symbols=SYMBOLS)

def view_prediction(i):
    print(input_strings[i][::-1].strip('_'))
    print("--------")
    print("Targ:", target_strings[i].strip('_'))
    print("Pred:", pred_strings[i].strip('_'))
a = interact(view_prediction, i=(0, batch_size - 1))

a=-5
if a<-3:
  a-=-29
  if a>-34:
    a-=-1
print(a)
--------
Targ: 25
Pred: 25
